In [90]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
from scipy import stats

In [91]:
data=np.load("eeg_22_channel_data.npz")

input=data["input"].astype(np.float64)
targets=data["targets"].astype(np.int32)

for i in range(targets.shape[0]):
    targets[i]=targets[i]-7

np.unique(targets)

array([0, 1, 2, 3])

In [92]:
# feature extraction from eeg data
def mean(x):
    return(np.mean(x,axis=-1))

def std(x):
    return(np.std(x,axis=-1))

def ptp(x):
    return(np.ptp(x,axis=-1))

def vari(x):
    return(np.var(x,axis=-1))

def minim(x):
    return(np.min(x,axis=-1))

def maxi(x):
    return(np.max(x,axis=-1))

def argmaxim(x):  ##pos of max
    return(np.argmax(x,axis=-1))

def argmini(x):
    return(np.argmin(x,axis=-1))

def rms(x):
    return(np.sqrt(np.mean(x**2,axis=-1)))

def mean(x):
    return(np.mean(x,axis=-1))

def abs_diff_signal(x):
    return(np.sum(np.abs(np.diff(x,axis=-1)),axis=-1))

def kurtosis(x):
    return(stats.kurtosis(x,axis=-1))

def skewness(x):
    return(stats.skew(x,axis=-1))

In [93]:
def concatenate_features(x):
    return(np.concatenate((mean(x),std(x),ptp(x),vari(x),minim(x),maxi(x),argmaxim(x) ,
                           argmini(x),rms(x), mean(x),abs_diff_signal(x),kurtosis(x),skewness(x)),axis=1))

In [94]:
input_features=concatenate_features(input)
input_features.shape,targets.shape

((2448, 286), (2448,))

In [95]:
# int(input_features.shape[0]*0.8),int(targets.shape[0]*0.8)

In [96]:
input_features=input_features[:int(input_features.shape[0]*0.8)]
input_targets=targets[:int(targets.shape[0]*0.8)]

print(input_features.shape)
print(input_targets.shape)


validation_input=input_features[int(input_features.shape[0]*0.8):int(input_features.shape[0]*0.9)]
validation_targets=targets[int(input_features.shape[0]*0.8):int(input_features.shape[0]*0.9)]

print(validation_input.shape)
print(validation_targets.shape)

test_feature=input_features[int(input_features.shape[0]*0.9):]
test_targets=targets[int(input_features.shape[0]*0.9):int(input_features.shape[0])]

test_targets.shape , test_feature.shape

(1958, 286)
(1958,)
(196, 286)
(196,)


((196,), (196, 286))

In [97]:
input_size=1958*286
output_size=4
hidden_layer_size=50

model=tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size,activation="relu"),
                            tf.keras.layers.Dense(hidden_layer_size,activation="relu"),
                            tf.keras.layers.Dense(hidden_layer_size,activation="relu"),
                            tf.keras.layers.Dense(hidden_layer_size,activation="relu"),
                            tf.keras.layers.Dense(output_size,activation="softmax")
])


In [99]:
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"])

input_features.shape

(1958, 286)

In [103]:
num_epoch=100

model.fit(input_features,input_targets,
          batch_size=100,
          epochs=num_epoch,
          validation_data=(validation_input,validation_targets),verbose=2,
          callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)]
          
          )


Epoch 1/100
20/20 - 0s - loss: 0.8385 - accuracy: 0.6691 - val_loss: 0.8004 - val_accuracy: 0.6990 - 193ms/epoch - 10ms/step
Epoch 2/100
20/20 - 0s - loss: 0.8431 - accuracy: 0.6553 - val_loss: 0.7774 - val_accuracy: 0.7041 - 112ms/epoch - 6ms/step
Epoch 3/100
20/20 - 0s - loss: 0.8156 - accuracy: 0.6726 - val_loss: 0.7782 - val_accuracy: 0.6837 - 103ms/epoch - 5ms/step
Epoch 4/100
20/20 - 0s - loss: 0.8467 - accuracy: 0.6496 - val_loss: 0.7676 - val_accuracy: 0.6939 - 88ms/epoch - 4ms/step
Epoch 5/100
20/20 - 0s - loss: 0.8254 - accuracy: 0.6573 - val_loss: 0.7531 - val_accuracy: 0.7296 - 99ms/epoch - 5ms/step
Epoch 6/100
20/20 - 0s - loss: 0.7814 - accuracy: 0.6834 - val_loss: 0.7673 - val_accuracy: 0.7092 - 87ms/epoch - 4ms/step
Epoch 7/100
20/20 - 0s - loss: 0.7543 - accuracy: 0.6997 - val_loss: 0.7140 - val_accuracy: 0.7398 - 85ms/epoch - 4ms/step
Epoch 8/100
20/20 - 0s - loss: 0.7717 - accuracy: 0.6936 - val_loss: 0.8695 - val_accuracy: 0.6531 - 87ms/epoch - 4ms/step
Epoch 9/100


In [104]:
model.evaluate(test_feature,test_targets)

7/7 [==============================] - 0s 2ms/step - loss: 0.6142 - accuracy: 0.7959


[0.6141600608825684, 0.795918345451355]